In [19]:

import pandas as pd
import pickle
import os
import numpy as np


In [13]:

def getRecordsLocal():
    location = '../data/wisebeat/raw_data_storage/sample_records.pkl'
    with open(location, 'rb') as file:
        data = pickle.load(file)
        
    return data
    

In [16]:
dataset = pd.DataFrame(getRecordsLocal())

In [17]:
dataset




,names,locations,types
0,WRLD-Kick-1-D.wav,/Users/vilhjamurr.vilhjalmsson/Desktop/Project...,KICK
1,WRLD-Kick-3-G.wav,/Users/vilhjamurr.vilhjalmsson/Desktop/Project...,KICK
2,WRLD-Kick-5-A.wav,/Users/vilhjamurr.vilhjalmsson/Desktop/Project...,KICK
3,WRLD-Kick-2-F.wav,/Users/vilhjamurr.vilhjalmsson/Desktop/Project...,KICK
4,WRLD-Kick-6-B.wav,/Users/vilhjamurr.vilhjalmsson/Desktop/Project...,KICK
5,WRLD-Kick-4-G.wav,/Users/vilhjamurr.vilhjalmsson/Desktop/Project...,KICK


In [ ]:

# Setja upp label encoder á types


In [20]:

class Config(object):
    def __init__(self,
                 sampling_rate=16000, audio_duration=2, n_classes=41,
                 use_mfcc=True, n_folds=10, learning_rate=0.0001, 
                 max_epochs=50, n_mfcc=20):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_mfcc = use_mfcc
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs

        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        else:
            self.dim = (self.audio_length, 1)

config = Config()


In [ ]:

class FeatureConfig(object):
    def __init__(self, sampling_rate=16000, audio_duration=1, mfcc_mels=20):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.mfcc_mels = mfcc_mels
        self.sample_length = self.sampling_rate * self.audio_duration
        self.sample_dimension = (self.mfcc_mels, 1 + int(np.floor(self.sample_length/512)), 1)



In [ ]:

def prepare_data(df, config, data_dir):
    X = np.empty(shape=(df.shape[0], config.dim[0], config.dim[1], 1))
    input_length = config.audio_length
    for i, fname in enumerate(df.index):
        #print(df['fname'])
        file_path = data_dir + df['fname'][i]
        data, _ = librosa.core.load(file_path, sr=config.sampling_rate, res_type="kaiser_fast")

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, input_length - len(data) - offset), "constant")

        data = librosa.feature.mfcc(data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
        data = np.expand_dims(data, axis=-1)
        X[i,] = data
    return X


In [ ]:

def prepare_audiofeatures(df, mfcc_config):
    features = np.empty(shape=(df.shape[0], config.dim[0], config.dim[1], 1))
    
